# MPS Hamiltonian Learning with TensorKrowch

In [4]:
import numpy as np
import torch
from torchvision import transforms, datasets
import tensorkrowch as tk

import pandas as pd
import glob
import yaml

### Things to do:

This will be a pytorch version of the Hamiltonian learning problem, where the NN has a layer with an MPS structure from tensorkrowch. There are a few things that I need to sort out:
1. This NN doesn't train from data, it starts from an ansatz and modifies it until it finds the optimal solution
2. I still need to run the dynamics for every epoch, which consist on:
    2.1. Taking initial state and applyting rotations in the X,Y and Z directions, with the option to customize which rotations I want to apply
    2.2. Doing a time evolution of the resulting state under a Hamiltonian. The Hamiltonian contains only interaction terms, and it also must be customizable
3. After running the Hamiltonian, we extract bitstring probabilities and compute nll loss function with the input data, which are the generated bitstrings 

## Data loading

In [5]:
def load_config(config_path):
    '''Load configuration from YAML file'''
    with open(config_path, 'r') as file:
        config = yaml.safe_load(file)
    return config

def load_experimental_data(config):
    """Load experimental/simulated data"""
    N = config["L"]
    chi = config['bond_dimension']
    T_max = config["t_max"]
    search_pattern = f"../data/experimental_data_quantum_sampling_L{N}_Chi_{chi}_*_counts.csv"
    files = glob.glob(search_pattern)

    if not files:
        raise FileNotFoundError(f"No data found for L={N}")

    config_file = files[0]
    file_core = config_file.replace(".csv", "").replace("../data/experimental_data_quantum_sampling_", "")
    
    print(f"\n{'='*60}")
    print(f"LOADING DATA: {file_core}")
    print(f"{'='*60}")
    
    df_counts = pd.read_csv(f"../data/experimental_data_quantum_sampling_{file_core}.csv")
        
    # Remove leading single quote if present
    if df_counts['bitstring'].astype(str).str.startswith("'").all():
        df_counts['bitstring'] = df_counts['bitstring'].str[1:]
    
    # Now extract values
    bitstrings = df_counts['bitstring'].values.astype(str)
    counts_shots = df_counts['count'].values.astype(np.int32)
    
    return bitstrings, counts_shots

## Useful functions

In [ ]:
def paulis(dtype=torch.complex64):
    '''Creates single-qubit basis operators'''
    sx = torch.tensor([[0., 1.], [1., 0.]], dtype=dtype)
    sy = torch.tensor([[0., -1j], [1j, 0.]], dtype=dtype)
    sz = torch.tensor([[1., 0.], [0., -1.]], dtype=dtype)
    id2 = torch.eye(2, dtype=dtype)
    return sx, sy, sz, id2

def kron_n(ops):
    '''Tensor product of a list of operators'''
    out = ops[0]
    for A in ops[1:]:
        out = torch.kron(out, A)
    return out

In [ ]:
def prepare_initial_state(L, kind):
    pass

In [ ]:
config_file = "/Users/omichel/Desktop/qilimanjaro/projects/retech/retech_2025/tensorkrowch_version/config/MPS_learning_configuration.yaml"

#load configuration
print(config_file)
CONFIG = load_config(config_file)

# Load data
bitstrings, counts_shots = load_experimental_data(CONFIG)

L = CONFIG['L']
inital_state_kind = CONFIG['initial_sate_kind']
dim = 2**L

psi0 = prepare_initial_state(L, inital_state_kind)

/Users/omichel/Desktop/qilimanjaro/projects/retech/retech_2025/tensorkrowch_version/config/MPS_learning_configuration.yaml

LOADING DATA: L4_Chi_5_R50000_counts
